In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb


/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [3]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [4]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [5]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

In [6]:
#cat_var = cat_var[1:]
#cat_var = cat_var[1:]
cat_var = cat_var + ['weightclass']

In [7]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [8]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [9]:
x_test = test_data.drop('hospital_death',axis=1)

In [10]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
# ### impute bmi values based on height and weight
# count = 0
# bmi_list = []
# for i in tqdm(range(x_comb.shape[0])):
    
#     if pd.isnull(x_comb["weight"].iloc[i]) or pd.isnull(x_comb["height"].iloc[i]):
#         bmi_list.append(x_comb["bmi"].iloc[i])
#         continue
#     else:
        
#         count += 1
#         bmi_list.append(x_comb["weight"].iloc[i] / ((x_comb["height"].iloc[i]/100) ** 2))

In [ ]:
#x_comb['bmi'] = bmi_list

In [11]:
x_comb['apache_2_bodysystem'] = x_comb['apache_2_bodysystem'].str.replace('Undefined Diagnoses', 
                                                                          'Undefined diagnoses', regex=False)

In [12]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] = x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [16]:
x_comb['hospital_admit_source'] = x_comb['hospital_admit_source'].replace({'Other ICU': 'ICU',
                                                                           'ICU to SDU':'SDU', 
                                                                           'Step-Down Unit (SDU)': 'SDU',        
                                                                           'Other Hospital':'Other',
                                                                           'Observation': 'Recovery Room',
                                                                           'Acute Care/Floor': 'Acute Care'})

In [17]:
x_comb['icu_type'] = x_comb['icu_type'].replace({'CCU-CTICU': 
                                                 'Grpd_CICU', 
                                                 'CTICU':'Grpd_CICU', 
                                                 'Cardiac ICU':'Grpd_CICU'})


In [13]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [18]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 32/32 [00:04<00:00,  6.92it/s]


In [19]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [20]:
x_comb["age"].mean()

62.00996491901006

In [22]:
for col in tqdm(new_cont_var):
    x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

100%|██████████| 153/153 [00:00<00:00, 523.99it/s]


In [23]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

32it [00:01, 17.44it/s]


In [24]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 71420.07it/s]


In [25]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [26]:
# #inter_cols3
# #x_comb['d1_heartrate_max_cat'] = 
# to_binned_columns = ['d1_heartrate_max','height','d1_hemaglobin_max','d1_hemaglobin_min','d1_glucose_max',
#                  'd1_glucose_min','d1_calcium_max','d1_calcium_min','d1_bilirubin_max','d1_bilirubin_min',
#                  'h1_temp_max','h1_temp_min','d1_sodium_max','d1_sodium_min','d1_potassium_max','d1_potassium_min',
#                  ]

# binned_cols = []

# for cols in tqdm(to_binned_columns) :
#     x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 8,labels=False))
#     binned_cols.append(cols+'_binned')


In [28]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'labs blood gas']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [ ]:
#binned_cols

In [ ]:
#train_data.columns[0:100]

list_imp_cols = ['icu_type','weightclass','age_category','gender','d1_heartrate_max_binned','height_binned']

inter_cols4 = []

for col1 in tqdm(list_imp_cols) :
    for col2 in list_imp_cols:
        if col1 !=col2 :
            x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
            inter_cols4.append(col1+col2)
            

In [ ]:
for cols in tqdm(inter_cols4):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
#train_data.columns
# list_groups = list(data_dict['Category'].unique())
# list_groups = list_groups[1:]

# grp_ls = []

# for grp in tqdm(list_groups):
    
#     grp_ls.append(list(data_dict[data_dict['Category'] == grp]['Variable Name']))

In [ ]:
# inter_cols5 = []

# for i in tqdm(range(0,len(grp_ls))):
#     p = grp_ls[i]
#     cols = list(set(p).intersection(set(x_comb.columns)))
#     #str_cols = '_'.join(cols)
    
#     x_comb['grp_cols'+str(i)] = x_comb[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
#     inter_cols5.append('grp_cols'+str(i))


In [ ]:
# for cols in tqdm(inter_cols5):
#     le = LabelEncoder()
#     x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
#Critical -way interaction

# inter_cols6 = []

x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)

x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)

x_comb['gender'+'d1_hemaglobin_max'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_hemaglobin_max_binned'].astype(str)
x_comb['gender'+'d1_hemaglobin_min'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_hemaglobin_min_binned'].astype(str)

x_comb['gender'+'d1_glucose_max'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_glucose_max_binned'].astype(str)
x_comb['gender'+'d1_glucose_min'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_glucose_min_binned'].astype(str)

x_comb['gender'+'d1_calcium_max'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_calcium_max_binned'].astype(str)
x_comb['gender'+'d1_calcium_min'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_calcium_min_binned'].astype(str)

x_comb['gender'+'d1_bilirubin_max'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_bilirubin_max_binned'].astype(str)
x_comb['gender'+'d1_bilirubin_min'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_bilirubin_min_binned'].astype(str)

x_comb['gender'+'h1_temp_max'] = x_comb['gender'].astype(str) + '_' + x_comb['h1_temp_max_binned'].astype(str)
x_comb['gender'+'h1_temp_min'] = x_comb['gender'].astype(str) + '_' + x_comb['h1_temp_max_binned'].astype(str)

x_comb['gender'+'d1_sodium_max'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_sodium_max_binned'].astype(str)
x_comb['gender'+'d1_sodium_min'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_sodium_min_binned'].astype(str)

x_comb['gender'+'d1_potassium_max'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_potassium_max_binned'].astype(str)
x_comb['gender'+'d1_potassium_min'] = x_comb['gender'].astype(str) + '_' + x_comb['d1_potassium_min_binned'].astype(str)


x_comb['age'+'h1_temp_max'] = x_comb['age'].astype(str) + '_' + x_comb['h1_temp_max'].astype(str)
x_comb['age'+'h1_temp_min'] = x_comb['age'].astype(str) + '_' + x_comb['h1_temp_min'].astype(str)


inter_cols6 = ['age'+'gender'+'weighted_classt',
               'age'+'gender'+'ethnicity',
               'gender'+'d1_hemaglobin_max',
               'gender'+'d1_glucose_max',
               'gender'+'d1_glucose_min',
               'gender'+'d1_calcium_max',
               'gender'+'d1_bilirubin_max',
              'gender'+'h1_temp_max',
               'age'+'h1_temp_max','gender'+'d1_hemaglobin_min',
               'gender'+'d1_calcium_min',
              'gender'+'d1_bilirubin_min',
               'gender'+'h1_temp_min',
               'age'+'h1_temp_min',
               'gender'+'d1_sodium_max',
              'gender'+'d1_sodium_min',
               'gender'+'d1_potassium_max',
               'gender'+'d1_potassium_min']



In [ ]:
#inter_cols6

In [ ]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
#final_cat_var = new_cat_var + count_var + ['age_category'] + inter_cols1 + inter_cols2 + inter_cols3 + inter_cols4


##### Target Encoding

In [ ]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']

In [ ]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

In [ ]:
x_comb.shape

In [ ]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [ ]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [ ]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count']
    

In [ ]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [ ]:
#new_cat_var

In [ ]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

In [ ]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [ ]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [ ]:
x_comb.shape

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [ ]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':8,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':16,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.25, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [ ]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [ ]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    10000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=2000,
                    verbose_eval=200
                    )

In [ ]:
pred = lgb_clf.predict(test_x)

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub40_4thFeb2020_cont_to_binned_vars.csv',index = False)

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=130, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 130000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 3000)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions / 10.0

In [ ]:
data_sub = x_test[['encounter_id']]

In [ ]:
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub60_10fold_4thFeb2020.csv',index = False)

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm1.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm2.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold1.csv')
# a6 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold.csv')
# a7 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub10_lgb_kfold2.csv')
# a8 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
# a9 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub11_ensemble_imp1.csv')
# a10 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub12_catb.csv')
a11 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_lgb_ensemble1_01022020.csv')
a12 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub4_ensemble1_01022020.csv')
a13 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
a14 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_10fold_4thFeb2020.csv')
a15 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm3.csv')
a16 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub6_10fold_4thFeb2020.csv')
a17 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub60_10fold_4thFeb2020.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (
#             a1['hospital_death'] + a2['hospital_death'] + a3['hospital_death'] + 
#             a4['hospital_death'] + a5['hospital_death'] + a6['hospital_death'] +
#             a7['hospital_death'] + a8['hospital_death'] + a9['hospital_death'] +
#             a10['hospital_death'] + 
            a11['hospital_death'] + a12['hospital_death'] +
            a13['hospital_death'] + a14['hospital_death'] + a15['hospital_death'] +
            a16['hospital_death'] + a17['hospital_death']
            
           )/7.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2_ensemble_5thFeb2020.csv',index = False)

In [ ]:
#train_x['encounter_id'].unique().shape


In [ ]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
sub_data

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

In [ ]:
x_test.head()

In [ ]:
ls

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2.csv',index = False)

In [ ]:
data_sub.shape